In [7]:
#import library
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [8]:
#load dataset yang akan digunakan untuk test
data = pd.read_csv('data\data.csv')

Mempersiapkan dataset untuk melakukan testing

In [9]:
data.head()

,text,labels
0,"According to a recent OFSTED report, a school ...",1
1,In Birmingham there is a school where girls an...,1
2,A school in Birningham is still separating gir...,1
3,The police and politicians are covering up the...,1
4,Muslims grooming gangs are protected by the go...,1


Menampilkan lima baris data pertama

In [10]:
testing = data['text'][685]

testing

"I figured it out by age 5. The good thing is children will have access to healthcare, information, medicine, treatment and support. None of which existed for transexuals when I was a little girl growing up in the 1970's and 80's."

Kode diatas mengambil data pada baris ke 1 untuk dilakukan testing

In [11]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(testing)
    
endpoint = "http://localhost:8080/v1/models/hatespeech-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Hate Speech" if prediction_value > 0.6 else "No Hate Speech"
else:
    result = "Error: No predictions found in response."
print(prediction_value)
print(result)

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /v1/models/hatespeech-prediction-model:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001260CC0C970>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

prepare_json() untuk mempersiapkan data input dalam json string yang akan digunakan untuk prediksi

Melakukan prediksi dari data testing yang dideploy pada api endpoint, apabila prediksi lebih besar dari 0.6 maka akan menghasilkan prediksi dari text "Hate Speech", apabila dibawah 0.6 maka "No Hate Speech"